# Setup

In [30]:
import pickle as pkl
from context_descriptor import ContextDescriptor
from prompt_generator import PromptGenerator
from model_request_handler import ModelRequestHandler
from open_ai_request_handler import OpenAiRequestHandler
import numpy as np
from examples_manager import Examples_manager
from sklearn.metrics import classification_report
import time

In [31]:
OPEN_AI_API_KEY = ""

In [32]:
DATASET = "EXTRASENSORY"
if DATASET not in ["DOMINO", "EXTRASENSORY"]:
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

In [33]:
extrasensory_classes = ['BICYCLING', 'LAYING_DOWN', 'MOVING_BY_CAR', 'ON_TRANSPORT', 'SITTING', 'STANDING', 'WALKING']
domino_classes = ['BRUSHING_TEETH', 'CYCLING', 'ELEVATOR_DOWN', 'ELEVATOR_UP', 'LYING', 'MOVING_BY_CAR', 'RUNNING', 'SITTING', 'SITTING_ON_TRANSPORT', 'STAIRS_DOWN', 'STAIRS_UP', 'STANDING', 'STANDING_ON_TRANSPORT', 'WALKING']

In [34]:
if DATASET == "DOMINO":
    CLASSES = domino_classes
elif DATASET == "EXTRASENSORY":
    CLASSES = extrasensory_classes
else:
    CLASSES = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

# Load dataset

In [35]:
if DATASET == "DOMINO":
    with open("datasets/domino/domino_unique_contexts.pkl", "rb") as file:
        unique_contexts = pkl.load(file)
    with open("datasets/domino/domino_unique_contexts_ontology_consistencies.pkl", "rb") as file:
        unique_contexts_ontology_consistencies = pkl.load(file)
    with open("datasets/domino/domino_unique_contexts_labels.pkl", "rb") as file:
        unique_contexts_labels = pkl.load(file)

elif DATASET == "EXTRASENSORY":
    with open("datasets/extrasensory/extrasensory_unique_contexts.pkl", "rb") as file:
        unique_contexts = pkl.load(file)
    with open("datasets/extrasensory/extrasensory_unique_contexts_labels.pkl", "rb") as file:
        unique_contexts_labels = pkl.load(file)
    with open("datasets/extrasensory/extrasensory_unique_contexts_ontology_consistencies.pkl", "rb") as file:
        unique_contexts_ontology_consistencies = pkl.load(file)
        
else:
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

# Define system message

In [36]:
system_message = f'''You are a domain expert in charge of checking which human activities are suitable with respect to the current surrounding context of a user. You will receive as input a description of the surrounding context of the user. You have to determine which are the activities that are suitable in the given context. This is the list of the possible user activities that you have to consider: {', '.join(CLASSES)}. Accomplish the task by following the steps below:Step 1: Analyze the context.Step 2: Determine which activities are likely in the given context, by analyzing each activity. Apply the open-world assumption: anything that is not explicit in the surrounding context of the user may be possible. Do not exclude activities for which you cannot determine their likelihood or the ones for which there are no specific information about objects and vehicles that the user is using.Step 3: Provide the result in the form of a list.'''

In [37]:
#Think about the "answer and step by step analyze each activity.
#system_message += "In the following i'll give you some examples of contexts that you can receive and the answer " \
#                  "that i would expect: "

# Initialize helper classes

In [38]:
contest_descriptor = ContextDescriptor()
prompt_generator = PromptGenerator(system_message)
#request_handler = ModelRequestHandler()
request_handler = OpenAiRequestHandler(api_key=OPEN_AI_API_KEY)

# Initialize example manager

In [43]:
if DATASET == "DOMINO":
    example_manager = Examples_manager("domino_examples.csv")
elif DATASET == "EXTRASENSORY":
    example_manager = Examples_manager("extrasensory_examples.csv")
else:
    example_manager = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

# Choose context desciption function

In [44]:
if DATASET == "DOMINO":
    create_context_description = lambda context_vector: contest_descriptor.create_domino_context_description(context_vector)
elif DATASET == "EXTRASENSORY":
    create_context_description = lambda context_vector: contest_descriptor.create_extrasensory_context_description(context_vector)
else:
    create_context_description = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

# Make the api calls

In [45]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fiori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fiori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
start_time = time.time()
all_responses = []
all_usages = []
i = 0
for c, l, o in list(zip(unique_contexts, unique_contexts_labels, unique_contexts_ontology_consistencies)):
    i += 1
    print(i, get_classes_from_binary_vector(o))
    #print(o, get_classes_from_binary_vector(o))
    description = create_context_description(c)
    #print(description)
    most_similar_examples =  example_manager.get_most_similar_examples(description)
    print('SIMILAR: \n', most_similar_examples) 
    #print(description)
    prompt = prompt_generator.generate_prompt(description, most_similar_examples)
    #print(prompt)
    responses, usages = request_handler.handle_request(prompt, repetitions=1, return_all=True)
    #print(responses)
    if [a.upper() for a in responses[0]] != get_classes_from_binary_vector(o) and [a.lower() for a in responses[0]] != get_classes_from_binary_vector(o):
        print()
        print(description, "\n", get_classes_from_binary_vector(o), "\n", responses[0])
    all_responses.append(responses)
    all_usages.append(usages)
    print("---------------------------------------------------------------------------------")
print("--- %s seconds ---" % (time.time() - start_time))

1 ['BICYCLING', 'WALKING']
MODEL RESPONSE:
['BICYCLING', 'WALKING']
---------------------------------------------------------------------------------
2 ['BICYCLING', 'LAYING_DOWN', 'SITTING', 'STANDING', 'WALKING']
MODEL RESPONSE:
['BICYCLING', 'LAYING_DOWN', 'SITTING', 'STANDING', 'WALKING']
---------------------------------------------------------------------------------
3 ['BICYCLING', 'MOVING_BY_CAR', 'WALKING']
MODEL RESPONSE:
['BICYCLING', 'MOVING_BY_CAR', 'WALKING']
---------------------------------------------------------------------------------
4 ['BICYCLING', 'MOVING_BY_CAR', 'WALKING']
MODEL RESPONSE:
['BICYCLING', 'MOVING_BY_CAR', 'WALKING']
---------------------------------------------------------------------------------
5 ['BICYCLING', 'MOVING_BY_CAR', 'WALKING']


In [ ]:
print()

# Dump data
Increment n each time to have different files

In [16]:
n = 40

## Dump answers

In [17]:
if DATASET == "DOMINO":
    answers_path = f"answers/domino/a_{n}.pkl"
elif DATASET == "EXTRASENSORY":
    answers_path = f"answers/extrasensory/a_{n}.pkl"
else:
    answers_path = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

In [18]:
with open(answers_path, "wb") as file:
    pkl.dump(all_responses, file)

# Dump usages

In [19]:
if DATASET == "DOMINO":
    usages_path = f"usages/domino/u_{n}.pkl"
elif DATASET == "EXTRASENSORY":
    usages_path = f"usages/extrasensory/u_{n}.pkl"
else:
    usages_path = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

In [20]:
with open(usages_path, "wb") as file:
    pkl.dump(all_usages, file)

# Load answers and ontology

In [21]:
if DATASET == "DOMINO":
    answers_path = f"answers/domino/a_{n}.pkl"
elif DATASET == "EXTRASENSORY":
    answers_path = f"answers/extrasensory/a_{n}.pkl"
else:
    answers_path = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

In [22]:
if DATASET == "DOMINO":
    ontology_path = f'datasets/domino/domino_unique_contexts_ontology_consistencies.pkl'
elif DATASET == "EXTRASENSORY":
    ontology_path = f"datasets/extrasensory/extrasensory_unique_contexts_ontology_consistencies.pkl"
else:
    answers_path = None
    raise ValueError("DATASET name must be 'DOMINO' or 'EXTRASENSORY'")

In [23]:
#with open(answers_path, "rb") as file:
#    all_responses = pkl.load(file)
with open(ontology_path, "rb") as file:
    ontology_consistencies = pkl.load(file)

# Compute results

In [24]:
def count_occurrences(activity, list_of_lists):
    count = 0
    for sublist in list_of_lists:
        count += sublist.count(activity)
    return count

In [25]:
def get_classes_from_binary_vector(bin_vec):
    classes_to_return = []
    for v, c in zip(bin_vec, CLASSES):
        if v==1:
            classes_to_return.append(c)
    return classes_to_return

In [26]:
min_count = 1

In [27]:
preds = []
for response in all_responses:
    # print(response)
    aggregated_response = []
    for act in CLASSES:
        if count_occurrences(act.lower(), response) >= min_count:
            aggregated_response.append(1)
        else:
            aggregated_response.append(0)
    preds.append(aggregated_response)

In [29]:
print(classification_report(ontology_consistencies[:64], preds, target_names=CLASSES))

               precision    recall  f1-score   support

    BICYCLING       0.81      0.40      0.54        55
  LAYING_DOWN       0.40      0.62      0.48        13
MOVING_BY_CAR       0.74      0.92      0.82        25
 ON_TRANSPORT       0.57      1.00      0.73         4
      SITTING       0.43      1.00      0.60        13
     STANDING       0.43      1.00      0.60        13
      WALKING       0.73      0.98      0.84        42

    micro avg       0.62      0.75      0.68       165
    macro avg       0.59      0.84      0.66       165
 weighted avg       0.68      0.75      0.67       165
  samples avg       0.69      0.78      0.68       165


In [68]:
CLASSES

['BICYCLING',
 'LAYING DOWN',
 'MOVING BY CAR',
 'ON TRANSPORT',
 'SITTING',
 'STANDING',
 'WALKING']

In [59]:
for r, c, p in zip(all_responses, unique_contexts_ontology_consistencies, preds):
    print(r, get_classes_from_binary_vector(p), get_classes_from_binary_vector(c))

[['bicycling', 'walking']] ['WALKING'] ['BICICLYNG', 'WALKING']
[['laying down', 'sitting', 'standing', 'walking']] ['SITTING', 'STANDING', 'WALKING'] ['BICICLYNG', 'LAYING_DOWN', 'SITTING', 'STANDING', 'WALKING']
[['bicycling', 'moving by car', 'walking']] ['WALKING'] ['BICICLYNG', 'MOVING_BY_CAR', 'WALKING']
[['bicycling', 'walking']] ['WALKING'] ['BICICLYNG', 'MOVING_BY_CAR', 'WALKING']
[['bicycling', 'moving by car', 'walking']] ['WALKING'] ['BICICLYNG', 'MOVING_BY_CAR', 'WALKING']
[['bicycling', 'laying down', 'sitting', 'standing', 'walking']] ['SITTING', 'STANDING', 'WALKING'] ['BICICLYNG', 'LAYING_DOWN', 'SITTING', 'STANDING', 'WALKING']
[['bicycling', 'laying down', 'sitting', 'standing', 'walking']] ['SITTING', 'STANDING', 'WALKING'] ['BICICLYNG', 'WALKING']
[['bicycling', 'walking']] ['WALKING'] ['BICICLYNG', 'MOVING_BY_CAR', 'WALKING']
[['bicycling', 'moving by car', 'walking']] ['WALKING'] ['BICICLYNG', 'MOVING_BY_CAR', 'WALKING']
[['laying down', 'sitting', 'standing', 'w

# Get results for DL

In [42]:
#TODO